In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy

# Load the spaCy model for English
nlp = spacy.load('en_core_web_sm')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Load the dataset
file_path = '/dataset/dataset_realtor.csv'
df = pd.read_csv(file_path)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:


# Rename columns
df.rename(columns={
    'msl_number': 'MSL Numbers',
    'bathroom_total': 'Bathrooms',
    'public_remarks': 'Public Remarks',
    'bedrooms': 'Bedrooms',
    'stories_total': 'Storey',
    'building_type': 'Building Types',
    'floor_area_measurements': 'Floor Area',
    'size_interior': 'Interior Size',
    'ammenities': 'Inhouse Ammenities',
    'contact_1': 'Primary Contact',
    'contact_2': 'Secondary Contact',
    'price': 'Price',
    'property_type': 'Property Type',
    'address': 'Address',
    'longitude': 'Longitude',
    'latitude': 'Latitude',
    'parking': 'Parking Type',
    'parking_space_total': 'Parking Space',
    'ownership_type': 'Ownership Type',
    'ammenities_nearBy': 'Nearby Amenities'
}, inplace=True)

In [ ]:
# Check the datatypes
df.dtypes

MSL Numbers            object
Bathrooms             float64
Public Remarks         object
Bedrooms               object
Storey                float64
Building Types         object
Floor Area             object
Interior Size          object
Inhouse Ammenities     object
Primary Contact        object
Secondary Contact      object
Price                  object
Property Type          object
Address                object
Longitude             float64
Latitude              float64
Parking Type           object
Parking Space         float64
Ownership Type         object
Nearby Amenities       object
dtype: object

In [ ]:
# Basic statistics
df.describe(include='all')

,MSL Numbers,Bathrooms,Public Remarks,Bedrooms,Storey,Building Types,Floor Area,Interior Size,Inhouse Ammenities,Primary Contact,Secondary Contact,Price,Property Type,Address,Longitude,Latitude,Parking Type,Parking Space,Ownership Type,Nearby Amenities
count,508,503.000000,508,503,360.000000,499,282,282,93,508,340,502,508,508,508.000000,508.000000,373,340.000000,500,300
unique,506,NaN,506,28,NaN,7,259,271,46,247,178,301,2,504,NaN,NaN,18,NaN,6,127
top,X8393968,NaN,"Welcome to 48 Connaught Avenue North, Hamilton...",3 + 0,NaN,House,472 sqft,472 sqft,"Exercise Centre, Party Room",519-542-9999,877-542-9992,"$ 3,49,900.00",Single Family,"48 CONNAUGHT AVENUE|Hamilton, Ontario L8L6Y8",NaN,NaN,Attached Garage,NaN,Freehold,"Park, Playground"
freq,2,NaN,2,80,NaN,274,3,3,12,18,16,9,503,2,NaN,NaN,191,NaN,315,22
mean,NaN,2.453280,NaN,NaN,2.026389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-87.762859,44.681338,NaN,3.435294,NaN,NaN
std,NaN,1.232569,NaN,NaN,2.067118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.091554,2.519738,NaN,2.712136,NaN,NaN
min,NaN,0.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-123.390916,42.959468,NaN,0.000000,NaN,NaN
25%,NaN,2.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-82.372305,43.202562,NaN,1.000000,NaN,NaN
50%,NaN,2.000000,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-80.250751,43.485457,NaN,3.000000,NaN,NaN
75%,NaN,3.000000,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-79.676013,44.257927,NaN,4.000000,NaN,NaN


In [ ]:
# Check for duplicates
df.duplicated().sum()

2

In [ ]:
# Handle duplicates by removing them
df_no_duplicates = df.drop_duplicates()

## MSL NUMBERS

In [ ]:
# Check for null values in MSL Numbers
null_msl_numbers = df['MSL Numbers'].isnull().sum()
print(f'Null MSL Numbers: {null_msl_numbers}')

Null MSL Numbers: 0


In [ ]:
# Check for uniqueness of MSL Numbers
unique_msl_numbers = df['MSL Numbers'].nunique()
total_msl_numbers = df['MSL Numbers'].count()
print(f'Total MSL Numbers: {total_msl_numbers}')
print(f'Unique MSL Numbers: {unique_msl_numbers}')

# If there are null values or non-unique MSL Numbers, we need to handle them
if null_msl_numbers > 0 or unique_msl_numbers != total_msl_numbers:
    print("There are issues with the MSL Numbers column that need to be addressed.")
else:
    print("MSL Numbers column is clean.")

Total MSL Numbers: 508
Unique MSL Numbers: 506
There are issues with the MSL Numbers column that need to be addressed.


In [ ]:
# Identify duplicate MSL Numbers and display them
duplicate_msl = df[df.duplicated(subset=['MSL Numbers'], keep=False)]
duplicate_msl_numbers = duplicate_msl['MSL Numbers'].unique()
duplicate_msl_entries = df[df['MSL Numbers'].isin(duplicate_msl_numbers)]

In [ ]:
print(duplicate_msl_entries)

    MSL Numbers  Bathrooms                                     Public Remarks  \
4      C8394548        1.0  Stop searching!! Your lovely home is waiting! ...   
94     X8393968        1.0  Welcome to 48 Connaught Avenue North, Hamilton...   
99     X8393968        1.0  Welcome to 48 Connaught Avenue North, Hamilton...   
100    C8394548        1.0  Stop searching!! Your lovely home is waiting! ...   

    Bedrooms  Storey Building Types Floor Area Interior Size  \
4          1     NaN      Apartment        NaN           NaN   
94         3     1.5          House        NaN           NaN   
99         3     1.5          House        NaN           NaN   
100        1     NaN      Apartment        NaN           NaN   

                                    Inhouse Ammenities Primary Contact  \
4    Security/Concierge, Exercise Centre, Party Roo...    888-884-0105   
94                                                 NaN    416-975-5588   
99                                                 

In [ ]:
# Drop duplicates by keeping the first occurrence
df_no_duplicates = df.drop_duplicates(subset=['MSL Numbers'], keep='first')

In [ ]:
# Verify that the duplicates have been handled
unique_msl_numbers_after = df_no_duplicates['MSL Numbers'].nunique()
total_msl_numbers_after = df_no_duplicates['MSL Numbers'].count()

In [ ]:
print(f'Total MSL Numbers after removing duplicates: {total_msl_numbers_after}')
print(f'Unique MSL Numbers after removing duplicates: {unique_msl_numbers_after}')

Total MSL Numbers after removing duplicates: 506
Unique MSL Numbers after removing duplicates: 506


## Bathroom Column

In [ ]:
# Check for missing values in Bathrooms
missing_bathrooms = df_no_duplicates['Bathrooms'].isnull().sum()
print(f'Missing Bathrooms: {missing_bathrooms}')

Missing Bathrooms: 5


In [ ]:
# Display the rows with missing Bathrooms
missing_bathrooms_rows = df_no_duplicates[df_no_duplicates['Bathrooms'].isnull()]
missing_bathrooms_rows

,MSL Numbers,Bathrooms,Public Remarks,Bedrooms,Storey,Building Types,Floor Area,Interior Size,Inhouse Ammenities,Primary Contact,Secondary Contact,Price,Property Type,Address,Longitude,Latitude,Parking Type,Parking Space,Ownership Type,Nearby Amenities
63,H4195846,NaN,Located in the heart of Mount Hope. Excellent ...,NaN,NaN,NaN,NaN,NaN,NaN,905-575-5478,905-575-7217,"$ 15,95,000.00",Vacant Land,"3117 HOMESTEAD Drive|Hamilton, Ontario L0R1W0",-79.913499,43.157855,NaN,NaN,NaN,NaN
69,X8394912,NaN,Located in the heart of Mount Hope. Excellent ...,NaN,NaN,NaN,-,1 ac,NaN,905-639-7676,905-681-9908,"$ 15,95,000.00",Vacant Land,"3117 HOMESTEAD DRIVE|Hamilton, Ontario L0R1W0",-79.913564,43.157996,NaN,NaN,NaN,Highway
262,40598346,NaN,This beautifully located building lot is set i...,NaN,NaN,NaN,NaN,NaN,NaN,613-382-0122,NaN,"$ 5,00,000.00",Vacant Land,"4 STARR Place|Kingston, Ontario K7L4V1",-76.402976,44.249559,NaN,NaN,Freehold,"Airport, Hospital, Schools"
267,40598112,NaN,Ready-to-go building lot in an established sec...,NaN,NaN,NaN,NaN,NaN,NaN,613-546-4208,NaN,"$ 3,25,000.00",Vacant Land,"1116 WOODHAVEN Drive|Kingston, Ontario K7P0R7",-76.594702,44.268610,NaN,NaN,Freehold,NaN
270,40597770,NaN,Rare opportunity to purchase a residentially z...,NaN,NaN,NaN,NaN,NaN,NaN,613-384-1997,NaN,"$ 1,89,900.00",Vacant Land,"622 BAGOT Street|Kingston, Ontario K7K3E5",-76.485908,44.240787,NaN,NaN,Freehold,"Hospital, Public Transit, Schools"


## Bedroom Column

In [ ]:
# Extract numerical values from the Bedrooms column and sum them up
def extract_bedrooms(value):
    if isinstance(value, str):
        nums = re.findall(r'\d+', value)
        return sum(map(int, nums)) if nums else None
    return value

In [ ]:
df['Bedrooms'] = df['Bedrooms'].apply(extract_bedrooms)

In [ ]:
# Check the result
df['Bedrooms'].head(10)

0    2.0
1    2.0
2    4.0
3    3.0
4    1.0
5    2.0
6    2.0
7    2.0
8    1.0
9    1.0
Name: Bedrooms, dtype: float64

In [ ]:
df.head(5)

,MSL Numbers,Bathrooms,Public Remarks,Bedrooms,Storey,Building Types,Floor Area,Interior Size,Inhouse Ammenities,Primary Contact,Secondary Contact,Price,Property Type,Address,Longitude,Latitude,Parking Type,Parking Space,Ownership Type,Nearby Amenities
0,40598830,2.0,Discover The Perfect Blend Of Artistic Design ...,2.0,2.0,Apartment,2807 sqft,2807,"Guest Suite, Party Room",905-858-3434,905-858-2682,"$ 6,79,900.00",Single Family,"1 AVONDALE Avenue Unit# 904|Toronto, Ontario M...",-79.409238,43.758162,Underground,1.0,Condominium,Public Transit
1,W8394664,2.0,"This beautifully maintained 2 bed, 2 bath, cor...",2.0,NaN,Apartment,NaN,NaN,"Security/Concierge, Exercise Centre, Sauna, St...",416-966-0300,416-966-0080,"$ 6,59,999.00",Single Family,"1532 - 165 LEGION ROAD N|Toronto, Ontario M8Y0B3",-79.488010,43.624121,NaN,NaN,Condominium/Strata,"Marina, Park, Public Transit"
2,E8394474,4.0,Presenting a stunning three-story townhouse lo...,4.0,3.0,Row / Townhouse,NaN,NaN,NaN,905-475-4750,905-475-4770,"$ 9,99,000.00",Single Family,"5 - 1251 BRIDLETOWNE CIRCLE|Toronto, Ontario M...",-79.311742,43.793613,NaN,NaN,Freehold,"Park, Public Transit, Schools"
3,C8394534,2.0,"A 1,240 Sq Ft 3 Bedroom, 2 Bedroom Luxury Cond...",3.0,NaN,Apartment,NaN,NaN,"Exercise Centre, Party Room, Security/Concierg...",647-797-1133,NaN,"$ 13,99,000.00",Single Family,"2906 - 330 RICHMOND STREET W|Toronto, Ontario ...",-79.392792,43.648931,Underground,1.0,Condominium/Strata,Public Transit
4,C8394548,1.0,Stop searching!! Your lovely home is waiting! ...,1.0,NaN,Apartment,NaN,NaN,"Security/Concierge, Exercise Centre, Party Roo...",888-884-0105,888-884-0106,"$ 5,29,000.00",Single Family,"1008 - 32 TROLLEY CRESCENT|Toronto, Ontario M5...",-79.355227,43.656756,NaN,NaN,Condominium/Strata,"Park, Public Transit, Schools"


In [ ]:
# Check for missing values in Bedrooms
missing_bedrooms = df['Bedrooms'].isnull().sum()
print(f'Missing Bedrooms: {missing_bedrooms}')

Missing Bedrooms: 5


In [ ]:

# Display the unique values in the Bedrooms column
unique_bedrooms = df['Bedrooms'].unique()
print("Unique values in Bedrooms column:\n", unique_bedrooms)

Unique values in Bedrooms column:
 [ 2.  4.  3.  1.  6.  5. nan  8.  7.  0.]


In [ ]:
# Check for non-numeric values or inconsistencies in Bedrooms
non_numeric_bedrooms = df[~df['Bedrooms'].apply(lambda x: isinstance(x, (int, float)))]
print("Non-numeric or inconsistent values in Bedrooms column:\n", non_numeric_bedrooms)

Non-numeric or inconsistent values in Bedrooms column:
 Empty DataFrame
Columns: [MSL Numbers, Bathrooms, Public Remarks, Bedrooms, Storey, Building Types, Floor Area, Interior Size, Inhouse Ammenities, Primary Contact, Secondary Contact, Price, Property Type, Address, Longitude, Latitude, Parking Type, Parking Space, Ownership Type, Nearby Amenities]
Index: []


In [ ]:
# Check for negative values in Bedrooms
negative_bedrooms = df[df['Bedrooms'] < 0]
print("Negative values in Bedrooms column:\n", negative_bedrooms)

Negative values in Bedrooms column:
 Empty DataFrame
Columns: [MSL Numbers, Bathrooms, Public Remarks, Bedrooms, Storey, Building Types, Floor Area, Interior Size, Inhouse Ammenities, Primary Contact, Secondary Contact, Price, Property Type, Address, Longitude, Latitude, Parking Type, Parking Space, Ownership Type, Nearby Amenities]
Index: []


In [ ]:
# Check the data type of the Bedrooms column
bedrooms_dtype = df['Bedrooms'].dtype
print(f'Data type of Bedrooms column: {bedrooms_dtype}')

Data type of Bedrooms column: float64


In [ ]:
# Display rows with missing Bedrooms
missing_bedrooms_rows = df[df['Bedrooms'].isnull()]
print("Rows with missing Bedrooms:\n", missing_bedrooms_rows)

Rows with missing Bedrooms:
     MSL Numbers  Bathrooms                                     Public Remarks  \
63     H4195846        NaN  Located in the heart of Mount Hope. Excellent ...   
69     X8394912        NaN  Located in the heart of Mount Hope. Excellent ...   
262    40598346        NaN  This beautifully located building lot is set i...   
267    40598112        NaN  Ready-to-go building lot in an established sec...   
270    40597770        NaN  Rare opportunity to purchase a residentially z...   

     Bedrooms  Storey Building Types Floor Area Interior Size  \
63        NaN     NaN            NaN        NaN           NaN   
69        NaN     NaN            NaN          -          1 ac   
262       NaN     NaN            NaN        NaN           NaN   
267       NaN     NaN            NaN        NaN           NaN   
270       NaN     NaN            NaN        NaN           NaN   

    Inhouse Ammenities Primary Contact Secondary Contact           Price  \
63               

In [ ]:
# Display rows with 0 Bedrooms
zero_bedrooms_rows = df[df['Bedrooms'] == 0]
print("Rows with 0 Bedrooms:\n", zero_bedrooms_rows)

Rows with 0 Bedrooms:
     MSL Numbers  Bathrooms                                     Public Remarks  \
134      965764        1.0  Follow your dream, Home! This an excellent opp...   
140      965777        1.0  Discover The Yates on Yates: A concrete & stee...   
162      965334        1.0  Modern living at its finest on Fort Streetï¿½s...   
177      965218        1.0  You wonï¿½t want to miss this one in the Jukeb...   
263    40598376        0.0  Unparalleled and distinguished, Whitney Manor ...   
274    40597268        1.0  This well maintained sweet condo is convenient...   

     Bedrooms  Storey Building Types Floor Area Interior Size  \
134       0.0     NaN      Apartment   415 sqft      415 sqft   
140       0.0     NaN      Apartment   360 sqft      360 sqft   
162       0.0     NaN      Apartment   571 sqft      571 sqft   
177       0.0     NaN      Apartment   442 sqft      442 sqft   
263       0.0     2.5          House     1 sqft             1   
274       0.0     1

## Storey

In [ ]:

# Check for missing values in Storey
missing_storey = df['Storey'].isnull().sum()
print(f'Missing Storey: {missing_storey}')

Missing Storey: 148


In [ ]:
# Display the unique values in the Storey column
unique_storey = df['Storey'].unique()
print("Unique values in Storey column:\n", unique_storey)

Unique values in Storey column:
 [ 2.   nan  3.   1.   1.5  2.5 15.   8.  14.   4.   9.   7.  18.  27.
  5.  10. ]


In [ ]:
# Check for non-numeric values or inconsistencies in Storey
non_numeric_storey = df[~df['Storey'].apply(lambda x: isinstance(x, (int, float)))]
print("Non-numeric or inconsistent values in Storey column:\n", non_numeric_storey)

Non-numeric or inconsistent values in Storey column:
 Empty DataFrame
Columns: [MSL Numbers, Bathrooms, Public Remarks, Bedrooms, Storey, Building Types, Floor Area, Interior Size, Inhouse Ammenities, Primary Contact, Secondary Contact, Price, Property Type, Address, Longitude, Latitude, Parking Type, Parking Space, Ownership Type, Nearby Amenities]
Index: []


In [ ]:
# Check for negative values in Storey
negative_storey = df[df['Storey'] < 0]
print("Negative values in Storey column:\n", negative_storey)

Negative values in Storey column:
 Empty DataFrame
Columns: [MSL Numbers, Bathrooms, Public Remarks, Bedrooms, Storey, Building Types, Floor Area, Interior Size, Inhouse Ammenities, Primary Contact, Secondary Contact, Price, Property Type, Address, Longitude, Latitude, Parking Type, Parking Space, Ownership Type, Nearby Amenities]
Index: []


In [ ]:
# Check the data type of the Storey column
storey_dtype = df['Storey'].dtype
print(f'Data type of Storey column: {storey_dtype}')

Data type of Storey column: float64


# Building Types Column

In [ ]:

# Check for missing values in Building Types
missing_building_types = df['Building Types'].isnull().sum()
print(f'Missing Building Types: {missing_building_types}')

Missing Building Types: 9


In [ ]:
# Display the unique values in the Building Types column
unique_building_types = df['Building Types'].unique()
print("Unique values in Building Types column:\n", unique_building_types)

Unique values in Building Types column:
 ['Apartment' 'Row / Townhouse' 'House' 'Triplex' nan 'Duplex'
 'Mobile Home' 'Other']


In [ ]:
# Check the data type of the Building Types column
building_types_dtype = df['Building Types'].dtype
print(f'Data type of Building Types column: {building_types_dtype}')

Data type of Building Types column: object


In [ ]:
# Standardize the naming convention for Building Types
df['Building Types'] = df['Building Types'].replace('Row / Townhouse', 'Townhouse')

In [ ]:
# Verify the changes
unique_building_types_after = df['Building Types'].unique()
unique_building_types_after

array(['Apartment', 'Townhouse', 'House', 'Triplex', nan, 'Duplex',
       'Mobile Home', 'Other'], dtype=object)

## Floor Area

In [ ]:
# Check for missing values in Floor Area
missing_floor_area = df['Floor Area'].isnull().sum()
print(f'Missing Floor Area: {missing_floor_area}')

Missing Floor Area: 226


In [ ]:
# Function to clean and convert Floor Area to numeric
def clean_floor_area(value):
    if isinstance(value, str):
        # Remove non-numeric characters
        value = re.sub(r'[^\d]', '', value)
        return float(value) if value else None
    return value

In [ ]:
# Apply the cleaning function
df['Floor Area'] = df['Floor Area'].apply(clean_floor_area)

In [ ]:
# Check the cleaned Floor Area values
cleaned_floor_area_unique = df['Floor Area'].unique()
print("Cleaned unique values in Floor Area column:\n", cleaned_floor_area_unique)

Cleaned unique values in Floor Area column:
 [2.8070e+03        nan 2.1400e+03 2.2340e+03 1.4000e+03 1.7120e+03
 1.1110e+03 4.5990e+03 8.0000e+02 5.5500e+02 1.0500e+03 1.1600e+03
 1.6560e+03 2.0150e+03 1.5830e+03 1.9270e+03 3.2110e+03 7.8200e+02
 1.9260e+03 1.0880e+03 1.5360e+03 1.3280e+03 2.3000e+03 1.9020e+03
 1.8440e+03 1.2390e+03 1.4100e+03 3.6930e+03 1.2960e+03 1.9350e+03
 2.4910e+03 1.7650e+03 6.9100e+02 1.6620e+03 1.8960e+03 8.8600e+02
 1.0820e+03 8.6800e+02 1.2230e+03 4.7200e+02 4.3000e+02 4.1500e+02
 1.7770e+03 4.2400e+02 8.7300e+02 5.1500e+02 2.0880e+03 3.6000e+02
 1.1570e+03 7.7000e+02 5.9900e+02 1.0690e+03 2.4380e+03 1.6980e+03
 1.9490e+03 1.7260e+03 1.1670e+03 1.3120e+03 3.5430e+03 7.9800e+02
 9.1900e+02 9.4500e+02 1.2120e+03 5.1700e+02 1.1510e+03 3.5910e+03
 1.0390e+03 8.2700e+02 1.1220e+03 5.7100e+02 5.8800e+02 5.8200e+02
 5.7300e+02 1.0130e+03 2.1410e+03 4.1240e+03 1.5460e+03 2.6540e+03
 4.2860e+03 6.9300e+02 1.0000e+03 2.1640e+03 9.7600e+02 4.4200e+02
 1.4200e+03 9.350

In [ ]:
# Check the data type of the Floor Area column
floor_area_dtype = df['Floor Area'].dtype
print(f'Data type of Floor Area column: {floor_area_dtype}')

Data type of Floor Area column: float64


In [ ]:
df.head(5)

,MSL Numbers,Bathrooms,Public Remarks,Bedrooms,Storey,Building Types,Floor Area,Interior Size,Inhouse Ammenities,Primary Contact,Secondary Contact,Price,Property Type,Address,Longitude,Latitude,Parking Type,Parking Space,Ownership Type,Nearby Amenities
0,40598830,2.0,Discover The Perfect Blend Of Artistic Design ...,2.0,2.0,Apartment,2807.0,2807,"Guest Suite, Party Room",905-858-3434,905-858-2682,"$ 6,79,900.00",Single Family,"1 AVONDALE Avenue Unit# 904|Toronto, Ontario M...",-79.409238,43.758162,Underground,1.0,Condominium,Public Transit
1,W8394664,2.0,"This beautifully maintained 2 bed, 2 bath, cor...",2.0,NaN,Apartment,NaN,NaN,"Security/Concierge, Exercise Centre, Sauna, St...",416-966-0300,416-966-0080,"$ 6,59,999.00",Single Family,"1532 - 165 LEGION ROAD N|Toronto, Ontario M8Y0B3",-79.488010,43.624121,NaN,NaN,Condominium/Strata,"Marina, Park, Public Transit"
2,E8394474,4.0,Presenting a stunning three-story townhouse lo...,4.0,3.0,Townhouse,NaN,NaN,NaN,905-475-4750,905-475-4770,"$ 9,99,000.00",Single Family,"5 - 1251 BRIDLETOWNE CIRCLE|Toronto, Ontario M...",-79.311742,43.793613,NaN,NaN,Freehold,"Park, Public Transit, Schools"
3,C8394534,2.0,"A 1,240 Sq Ft 3 Bedroom, 2 Bedroom Luxury Cond...",3.0,NaN,Apartment,NaN,NaN,"Exercise Centre, Party Room, Security/Concierg...",647-797-1133,NaN,"$ 13,99,000.00",Single Family,"2906 - 330 RICHMOND STREET W|Toronto, Ontario ...",-79.392792,43.648931,Underground,1.0,Condominium/Strata,Public Transit
4,C8394548,1.0,Stop searching!! Your lovely home is waiting! ...,1.0,NaN,Apartment,NaN,NaN,"Security/Concierge, Exercise Centre, Party Roo...",888-884-0105,888-884-0106,"$ 5,29,000.00",Single Family,"1008 - 32 TROLLEY CRESCENT|Toronto, Ontario M5...",-79.355227,43.656756,NaN,NaN,Condominium/Strata,"Park, Public Transit, Schools"


In [ ]:
df.tail(5)

,MSL Numbers,Bathrooms,Public Remarks,Bedrooms,Storey,Building Types,Floor Area,Interior Size,Inhouse Ammenities,Primary Contact,Secondary Contact,Price,Property Type,Address,Longitude,Latitude,Parking Type,Parking Space,Ownership Type,Nearby Amenities
503,A2138070,2.0,Welcome to this beautiful condo situated in on...,2.0,5.0,Apartment,1294.0,1294.29 sqft,"Exercise Centre, Recreation Centre",403-259-4141,NaN,"$ 5,05,000.00",Single Family,"104, 30 Discovery Ridge Close SW|Calgary, Albe...",-114.213513,51.013349,Underground,1.0,Condominium/Strata,"Park, Playground"
504,A2138440,2.0,Welcome to this open concept 3 bed (2+1) 2 bat...,3.0,2.0,House,1336.0,1336.31 sqft,NaN,403-252-5900,NaN,"$ 5,75,000.00",Single Family,"327 Copperfield Heights SE|Calgary, Alberta t2...",-113.945793,50.913724,NaN,NaN,Freehold,NaN
505,A2138203,1.0,Welcome to your dream home in the heart of Sig...,1.0,4.0,Apartment,726.0,725.51 sqft,Car Wash,403-618-6984,NaN,"$ 3,09,999.00",Single Family,"2104, 1888 Signature Park SW|Calgary, Alberta ...",-114.166401,51.036846,Other,1.0,Condominium/Strata,"Park, Playground, Recreation Nearby"
506,A2137678,1.0,"Welcome to a premier adult building, Silverwoo...",1.0,10.0,Apartment,674.0,674.15 sqft,"Party Room, Recreation Centre",403-216-1600,403-284-4923,"$ 1,99,900.00",Single Family,"710, 8604 48 Avenue NW|Calgary, Alberta T3B5E6",-114.211446,51.096366,Underground,1.0,Condominium/Strata,"Park, Playground"
507,A2135951,12.0,This splendid French Provence-inspired estate ...,6.0,2.0,House,10601.0,10600.59 sqft,NaN,403-615-6995,NaN,"$ 99,00,000.00",Single Family,"19 Pump Hill Close SW|Calgary, Alberta T2V5E5",-114.099000,50.965375,Garage,9.0,Freehold,"Park, Playground"


Dropping Interior Size column as it is same as Floor Area

In [ ]:
# Drop the Interior Size column
if 'Interior Size' in df.columns:
    df.drop(columns=['Interior Size'], inplace=True)

# Verify the column is dropped
df.columns

Index(['MSL Numbers', 'Bathrooms', 'Public Remarks', 'Bedrooms', 'Storey',
       'Building Types', 'Floor Area', 'Inhouse Ammenities', 'Primary Contact',
       'Secondary Contact', 'Price', 'Property Type', 'Address', 'Longitude',
       'Latitude', 'Parking Type', 'Parking Space', 'Ownership Type',
       'Nearby Amenities'],
      dtype='object')

## Contacts

In [ ]:


import re

# Check for missing values in Primary and Secondary Contact
missing_primary_contact = df['Primary Contact'].isnull().sum()
missing_secondary_contact = df['Secondary Contact'].isnull().sum()
print(f'Missing Primary Contact: {missing_primary_contact}')
print(f'Missing Secondary Contact: {missing_secondary_contact}')

Missing Primary Contact: 0
Missing Secondary Contact: 168


In [ ]:
# Function to clean and standardize phone numbers
def clean_phone_number(phone):
    if isinstance(phone, str):
        # Remove non-numeric characters
        phone = re.sub(r'\D', '', phone)
        # Format as xxx-xxx-xxxx if it has 10 digits
        if len(phone) == 10:
            phone = f"{phone[:3]}-{phone[3:6]}-{phone[6:]}"
        return phone
    return phone

# Apply the cleaning function to Primary and Secondary Contact
df['Primary Contact'] = df['Primary Contact'].apply(clean_phone_number)
df['Secondary Contact'] = df['Secondary Contact'].apply(clean_phone_number)
print("Cleaned Primary Contact values:\n", df['Primary Contact'].unique())
print("Cleaned Secondary Contact values:\n", df['Secondary Contact'].unique())

Cleaned Primary Contact values:
 ['905-858-3434' '416-966-0300' '905-475-4750' '647-797-1133'
 '888-884-0105' '905-305-1600' '416-628-1357' '905-712-9888'
 '416-391-3232' '416-962-1601' '905-909-0101' '416-924-3779'
 '416-686-1500' '905-268-1000' '416-918-4400' '416-883-0892'
 '905-665-2500' '905-604-7200' '905-305-0033' '905-459-7900'
 '905-604-1010' '905-278-3500' '866-530-7737' '905-882-6882'
 '416-322-8000' '705-601-0857' '416-289-3000' '416-360-0688'
 '905-201-9977' '416-745-2300' '905-678-8620' '905-513-8878'
 '416-443-0300' '905-822-6900' '416-490-1177' '905-364-0727'
 '905-554-0101' '416-282-3333' '416-989-6565' '416-236-1241'
 '416-691-3000' '647-360-8963' '905-940-4180' '416-533-5888'
 '416-366-8800' '905-508-9500' '905-475-3336' '905-575-9262'
 '416-975-5588' '905-831-2273' '905-518-7777' '905-575-5478'
 '905-828-1122' '905-648-4451' '905-385-9200' '905-639-7676'
 '905-270-2000' '888-966-3111' '905-648-3333' '905-573-1188'
 '905-945-1234' '905-842-7677' '289-389-1377' '905-5

## Price Column

In [ ]:
# Check for missing values in Price
missing_price = df['Price'].isnull().sum()
print(f'Missing Price: {missing_price}')

Missing Price: 6


In [ ]:
# Function to clean and convert Price to numeric
def clean_price(value):
    if isinstance(value, str):
        # Remove non-numeric characters
        value = re.sub(r'[^\d.]', '', value)
        return float(value) if value else None
    return value

In [ ]:
# Apply the cleaning function to Price
df['Price'] = df['Price'].apply(clean_price)

In [ ]:
# Check the data type of the Price column
price_dtype = df['Price'].dtype
print(f'Data type of Price column: {price_dtype}')

Data type of Price column: float64


In [ ]:
# Check the cleaned Price values
cleaned_price_unique = df['Price'].unique()
print("Cleaned unique values in Price column:\n", cleaned_price_unique)

Cleaned unique values in Price column:
 [ 679900.  659999.  999000. 1399000.  529000. 1088000.  624800.  445000.
  499900.  638000.  850000. 2200000.  630000. 9999000. 1200000.  629000.
  629999.  619800. 1400000.  548000.  925000.  696000. 1450000.  618000.
  669998. 1288000.  585880.  979000.  588000. 1699000.  708000.  699000.
  475000. 1299000.  628999.  799000. 1090000.  628000.  639990. 1799000.
  649000.  438800.  619000.  879900. 1199999.  476999. 1549000.  849000.
  859900.  518000. 1275000.  660000. 2500000. 3199000.  579000.  615000.
 1199000.  879000.  929000. 1595000.  965000. 1149900. 3995000.  428888.
  749900.  469777.  589900. 1680000.  818000.  699900.  689900.  785000.
  849999.  999900. 1499999. 1059900. 1160000. 1999999.  849900.  499999.
  349900.  844900.  974900.  799900. 1998900.  899900. 1399900. 1599900.
 1475000. 1199900.  859000. 1099000.  829900. 1149000.  375000. 1150000.
  640000.  745000.  549900.  419999.  519900.  489900.  469900.  335000.
  949900.  

In [ ]:
# Display basic statistics for the Price column
price_stats = df['Price'].describe()
print("Statistics for Price column:\n", price_stats)

Statistics for Price column:
 count    5.020000e+02
mean     9.877427e+05
std      9.406914e+05
min      1.488000e+05
25%      5.392250e+05
50%      7.299000e+05
75%      1.099000e+06
max      9.999000e+06
Name: Price, dtype: float64


## Property Type

In [ ]:
# Check for missing values in Property Type
missing_property_type = df['Property Type'].isnull().sum()
print(f'Missing Property Type: {missing_property_type}')

Missing Property Type: 0


In [ ]:
# Display the unique values in the Property Type column
unique_property_type = df['Property Type'].unique()
print("Unique values in Property Type column:\n", unique_property_type)

Unique values in Property Type column:
 ['Single Family' 'Vacant Land']


In [ ]:
# Check the data type of the Property Type column
property_type_dtype = df['Property Type'].dtype
print(f'Data type of Property Type column: {property_type_dtype}')

Data type of Property Type column: object


## Longitude and Latitude


In [ ]:
missing_longitude = df['Longitude'].isnull().sum()
missing_latitude = df['Latitude'].isnull().sum()
print(f'Missing Longitude: {missing_longitude}')
print(f'Missing Latitude: {missing_latitude}')

Missing Longitude: 0
Missing Latitude: 0


In [ ]:
longitude_dtype = df['Longitude'].dtype
latitude_dtype = df['Latitude'].dtype
print(f'Data type of Longitude column: {longitude_dtype}')
print(f'Data type of Latitude column: {latitude_dtype}')

Data type of Longitude column: float64
Data type of Latitude column: float64


## Parking Type


In [ ]:
missing_parking_type = df['Parking Type'].isnull().sum()
print(f'Missing Parking Type: {missing_parking_type}')

Missing Parking Type: 135


In [ ]:
unique_parking_type = df['Parking Type'].unique()
print("Unique values in Parking Type column:\n", unique_parking_type)

Unique values in Parking Type column:
 ['Underground' nan 'Garage' 'Attached Garage' 'Carport' 'Detached Garage'
 'No Garage' 'Gravel' 'Stall' 'Street' 'Open' 'Other' 'Parking Space(s)'
 'Surfaced' 'Visitor Parking' 'Interlocked' 'Concrete' 'Parking Pad'
 'Exposed Aggregate']


## Parking Space


In [ ]:
missing_parking_space = df['Parking Space'].isnull().sum()
print(f'Missing Parking Space: {missing_parking_space}')

Missing Parking Space: 168


In [ ]:
parking_space_dtype = df['Parking Space'].dtype
print(f'Data type of Parking Space column: {parking_space_dtype}')

Data type of Parking Space column: float64


## Ownership Type




In [ ]:
missing_ownership_type = df['Ownership Type'].isnull().sum()
print(f'Missing Ownership Type: {missing_ownership_type}')

Missing Ownership Type: 8


In [ ]:
unique_ownership_type = df['Ownership Type'].unique()
print("Unique values in Ownership Type column:\n", unique_ownership_type)

Unique values in Ownership Type column:
 ['Condominium' 'Condominium/Strata' 'Freehold' nan 'Leasehold' 'Strata'
 'Cooperative']


In [ ]:
# Check the data type of Ownership Type column
ownership_type_dtype = df['Ownership Type'].dtype
print(f'Data type of Ownership Type column: {ownership_type_dtype}')

Data type of Ownership Type column: object




---


## Feature Extraction from Public Remark (NUMERIC)

---



In [ ]:

# Check missing values before imputation for all relevant columns
missing_values_before = {
    'Bedrooms': df['Bedrooms'].isnull().sum(),
    'Bathrooms': df['Bathrooms'].isnull().sum(),
    'Storey': df['Storey'].isnull().sum(),
    'Floor Area': df['Floor Area'].isnull().sum(),
    'Price': df['Price'].isnull().sum(),
    'Parking Space': df['Parking Space'].isnull().sum()
}

In [ ]:
# Print missing values before imputation
for column, missing_count in missing_values_before.items():
    print(f'Missing {column} before imputation: {missing_count}')

Missing Bedrooms before imputation: 5
Missing Bathrooms before imputation: 5
Missing Storey before imputation: 148
Missing Floor Area before imputation: 227
Missing Price before imputation: 6
Missing Parking Space before imputation: 168


In [ ]:
def extract_numeric_feature_from_remarks(remarks, feature):
    if isinstance(remarks, str):
        remarks_lower = remarks.lower()  # Convert to lowercase for extraction
        if feature == 'bedrooms':
            match = re.search(r'(\d+)\s*bed(?:room)?s?', remarks_lower)
        elif feature == 'bathrooms':
            match = re.search(r'(\d+)\s*bat(?:hroom)?s?', remarks_lower)
        elif feature == 'storey':
            match = re.search(r'(\d+)\s*storey(?:s)?', remarks_lower) or re.search(r'(\d+)\s*stories', remarks_lower)
        elif feature == 'floor area':
            match = re.search(r'(\d+)\s*(?:sq\s*ft|sqft|sf|s\.f)', remarks_lower)
        elif feature == 'price':
            match = re.search(r'\$\s?(\d+(?:,\d{3})*(?:\.\d{2})?)', remarks_lower)
        elif feature == 'parking':
            match = re.search(r'(\d+)\s*parking\s*(?:space)?', remarks_lower)
        if match:
            return int(match.group(1).replace(',', '')) if feature != 'price' else float(match.group(1).replace(',', ''))
    return None

In [ ]:
# Apply the function to create new columns for extracted features
df['Extracted Bedrooms'] = df['Public Remarks'].apply(lambda x: extract_numeric_feature_from_remarks(x, 'bedrooms'))
df['Extracted Bathrooms'] = df['Public Remarks'].apply(lambda x: extract_numeric_feature_from_remarks(x, 'bathrooms'))
df['Extracted Storeys'] = df['Public Remarks'].apply(lambda x: extract_numeric_feature_from_remarks(x, 'storey'))
df['Extracted Floor Area'] = df['Public Remarks'].apply(lambda x: extract_numeric_feature_from_remarks(x, 'floor area'))
df['Extracted Price'] = df['Public Remarks'].apply(lambda x: extract_numeric_feature_from_remarks(x, 'price'))
df['Extracted Parking Space'] = df['Public Remarks'].apply(lambda x: extract_numeric_feature_from_remarks(x, 'parking'))


In [ ]:
# Display the first few rows to verify the changes
df[['Public Remarks', 'Extracted Bedrooms', 'Extracted Bathrooms']].head(10)

,Public Remarks,Extracted Bedrooms,Extracted Bathrooms
0,Discover The Perfect Blend Of Artistic Design ...,NaN,NaN
1,"This beautifully maintained 2 bed, 2 bath, cor...",2.0,2.0
2,Presenting a stunning three-story townhouse lo...,NaN,NaN
3,"A 1,240 Sq Ft 3 Bedroom, 2 Bedroom Luxury Cond...",3.0,NaN
4,Stop searching!! Your lovely home is waiting! ...,1.0,1.0
5,Welcome to the luxurious Tridel Building at Sk...,NaN,NaN
6,Spectacular Downtown Bloor St. Location! Bloo...,1.0,NaN
7,Welcome First Time Homebuyers and Investor whe...,NaN,NaN
8,"Luxury Condo By Daniels 'The Rockefeller ', On...",NaN,NaN
9,A boutique condo located at the heart of mid-t...,1.0,NaN


In [ ]:
df[['Public Remarks', 'Extracted Price', 'Extracted Floor Area']].head(10)

,Public Remarks,Extracted Price,Extracted Floor Area
0,Discover The Perfect Blend Of Artistic Design ...,NaN,NaN
1,"This beautifully maintained 2 bed, 2 bath, cor...",NaN,NaN
2,Presenting a stunning three-story townhouse lo...,4000.00,1845.0
3,"A 1,240 Sq Ft 3 Bedroom, 2 Bedroom Luxury Cond...",45000.00,240.0
4,Stop searching!! Your lovely home is waiting! ...,NaN,NaN
5,Welcome to the luxurious Tridel Building at Sk...,NaN,NaN
6,Spectacular Downtown Bloor St. Location! Bloo...,NaN,600.0
7,Welcome First Time Homebuyers and Investor whe...,61.77,NaN
8,"Luxury Condo By Daniels 'The Rockefeller ', On...",NaN,NaN
9,A boutique condo located at the heart of mid-t...,230.00,590.0


In [ ]:
df[['Public Remarks', 'Extracted Storeys', 'Extracted Parking Space']].head(10)

,Public Remarks,Extracted Storeys,Extracted Parking Space
0,Discover The Perfect Blend Of Artistic Design ...,NaN,NaN
1,"This beautifully maintained 2 bed, 2 bath, cor...",NaN,NaN
2,Presenting a stunning three-story townhouse lo...,NaN,NaN
3,"A 1,240 Sq Ft 3 Bedroom, 2 Bedroom Luxury Cond...",NaN,NaN
4,Stop searching!! Your lovely home is waiting! ...,NaN,NaN
5,Welcome to the luxurious Tridel Building at Sk...,NaN,2.0
6,Spectacular Downtown Bloor St. Location! Bloo...,NaN,NaN
7,Welcome First Time Homebuyers and Investor whe...,NaN,NaN
8,"Luxury Condo By Daniels 'The Rockefeller ', On...",NaN,NaN
9,A boutique condo located at the heart of mid-t...,NaN,NaN


In [ ]:
# Update the 'Bedrooms' column
df['Bedrooms'] = df.apply(
    lambda row: row['Extracted Bedrooms'] if pd.isnull(row['Bedrooms']) and not pd.isnull(row['Extracted Bedrooms']) else row['Bedrooms'],
    axis=1
)

# Update the 'Bathrooms' column
df['Bathrooms'] = df.apply(
    lambda row: row['Extracted Bathrooms'] if pd.isnull(row['Bathrooms']) and not pd.isnull(row['Extracted Bathrooms']) else row['Bathrooms'],
    axis=1
)

# Update the 'Storey' column
df['Storey'] = df.apply(
    lambda row: row['Extracted Storeys'] if pd.isnull(row['Storey']) and not pd.isnull(row['Extracted Storeys']) else row['Storey'],
    axis=1
)

# Update the 'Floor Area' column
df['Floor Area'] = df.apply(
    lambda row: row['Extracted Floor Area'] if pd.isnull(row['Floor Area']) and not pd.isnull(row['Extracted Floor Area']) else row['Floor Area'],
    axis=1
)

# Update the 'Price' column
df['Price'] = df.apply(
    lambda row: row['Extracted Price'] if pd.isnull(row['Price']) and not pd.isnull(row['Extracted Price']) else row['Price'],
    axis=1
)

# Update the 'Parking Space' column
df['Parking Space'] = df.apply(
    lambda row: row['Extracted Parking Space'] if pd.isnull(row['Parking Space']) and not pd.isnull(row['Extracted Parking Space']) else row['Parking Space'],
    axis=1
)

In [ ]:


# Check missing values after imputation
missing_values_after = {
    'Bedrooms': df['Bedrooms'].isnull().sum(),
    'Bathrooms': df['Bathrooms'].isnull().sum(),
    'Storey': df['Storey'].isnull().sum(),
    'Floor Area': df['Floor Area'].isnull().sum(),
    'Price': df['Price'].isnull().sum(),
    'Parking Space': df['Parking Space'].isnull().sum()
}


In [ ]:

# Print missing values after imputation
for column, missing_count in missing_values_after.items():
    print(f'Missing {column} after imputation: {missing_count}')


Missing Bedrooms after imputation: 3
Missing Bathrooms after imputation: 5
Missing Storey after imputation: 147
Missing Floor Area after imputation: 181
Missing Price after imputation: 6
Missing Parking Space after imputation: 167


In [ ]:

# Drop the extracted columns
df.drop(columns=['Extracted Bedrooms', 'Extracted Bathrooms', 'Extracted Storeys', 'Extracted Floor Area', 'Extracted Price', 'Extracted Parking Space'], inplace=True)


In [ ]:

# Display the first few rows to verify the changes
df[['Public Remarks', 'Bedrooms', 'Bathrooms', 'Storey', 'Floor Area', 'Price', 'Parking Space']].head(10)



,Public Remarks,Bedrooms,Bathrooms,Storey,Floor Area,Price,Parking Space
0,Discover The Perfect Blend Of Artistic Design ...,2.0,2.0,2.0,2807.0,679900.0,1.0
1,"This beautifully maintained 2 bed, 2 bath, cor...",2.0,2.0,NaN,NaN,659999.0,NaN
2,Presenting a stunning three-story townhouse lo...,4.0,4.0,3.0,1845.0,999000.0,NaN
3,"A 1,240 Sq Ft 3 Bedroom, 2 Bedroom Luxury Cond...",3.0,2.0,NaN,240.0,1399000.0,1.0
4,Stop searching!! Your lovely home is waiting! ...,1.0,1.0,NaN,NaN,529000.0,NaN
5,Welcome to the luxurious Tridel Building at Sk...,2.0,2.0,NaN,NaN,1088000.0,2.0
6,Spectacular Downtown Bloor St. Location! Bloo...,2.0,1.0,NaN,600.0,624800.0,1.0
7,Welcome First Time Homebuyers and Investor whe...,2.0,1.0,NaN,NaN,445000.0,1.0
8,"Luxury Condo By Daniels 'The Rockefeller ', On...",1.0,1.0,NaN,NaN,499900.0,1.0
9,A boutique condo located at the heart of mid-t...,1.0,1.0,NaN,590.0,638000.0,2.0




---


## Feature Extraction from Public Remark (CATEGORICAL)

---



In [ ]:
# Check for null values and unique values in specific columns
columns_to_check = ['Building Types', 'Parking Type', 'Ownership Type']

In [ ]:
for column in columns_to_check:
    missing_count = df[column].isnull().sum()
    unique_values = df[column].unique()
    print(f'Missing values in {column}: {missing_count}\n')
    print(f'Unique values in {column}: {unique_values}\n')

Missing values in Building Types: 9

Unique values in Building Types: ['Apartment' 'Townhouse' 'House' 'Triplex' nan 'Duplex' 'Mobile Home'
 'Other']

Missing values in Parking Type: 135

Unique values in Parking Type: ['Underground' nan 'Garage' 'Attached Garage' 'Carport' 'Detached Garage'
 'No Garage' 'Gravel' 'Stall' 'Street' 'Open' 'Other' 'Parking Space(s)'
 'Surfaced' 'Visitor Parking' 'Interlocked' 'Concrete' 'Parking Pad'
 'Exposed Aggregate']

Missing values in Ownership Type: 8

Unique values in Ownership Type: ['Condominium' 'Condominium/Strata' 'Freehold' nan 'Leasehold' 'Strata'
 'Cooperative']



In [ ]:


# Dictionary of keywords for each feature
keywords_dict = {
    'building_types': ['apartment', 'townhouse', 'house', 'triplex', 'duplex', 'mobile home', 'other'],
    'parking_type': ['underground', 'garage', 'attached garage', 'carport', 'detached garage', 'no garage', 'gravel', 'stall', 'none', 'street', 'open', 'other', 'parking space', 'surfaced', 'visitor parking', 'interlocked', 'concrete', 'parking pad', 'exposed aggregate'],
    'ownership_type': ['condominium', 'condominium/strata', 'freehold', 'leasehold', 'strata', 'cooperative']
}

In [ ]:


# Function to extract categorical features from Public Remarks dynamically
def extract_categorical_feature_from_remarks_dynamic(remarks, feature):
    if isinstance(remarks, str):
        remarks_lower = remarks.lower()  # Convert to lowercase for extraction
        if feature in keywords_dict:
            for keyword in keywords_dict[feature]:
                if keyword in remarks_lower:
                    return keyword.capitalize().replace(' ', '')
    return None

In [ ]:
# Apply the function to create new columns for extracted categorical features dynamically
for feature in keywords_dict.keys():
    df[f'Extracted_{feature}'] = df['Public Remarks'].apply(lambda x: extract_categorical_feature_from_remarks_dynamic(x, feature))


In [ ]:
# Update the original columns with extracted values
df['Building Types'] = df.apply(
    lambda row: row['Extracted_building_types'] if pd.isnull(row['Building Types']) and not pd.isnull(row['Extracted_building_types']) else row['Building Types'],
    axis=1
)

df['Parking Type'] = df.apply(
    lambda row: row['Extracted_parking_type'] if pd.isnull(row['Parking Type']) and not pd.isnull(row['Extracted_parking_type']) else row['Parking Type'],
    axis=1
)

df['Ownership Type'] = df.apply(
    lambda row: row['Extracted_ownership_type'] if pd.isnull(row['Ownership Type']) and not pd.isnull(row['Extracted_ownership_type']) else row['Ownership Type'],
    axis=1
)

In [ ]:
# Check missing values after imputation
missing_building_types_after = df['Building Types'].isnull().sum()
missing_parking_type_after = df['Parking Type'].isnull().sum()
missing_ownership_type_after = df['Ownership Type'].isnull().sum()
print(f'Missing Building Types after imputation: {missing_building_types_after}')
print(f'Missing Parking Type after imputation: {missing_parking_type_after}')
print(f'Missing Ownership Type after imputation: {missing_ownership_type_after}')

Missing Building Types after imputation: 7
Missing Parking Type after imputation: 50
Missing Ownership Type after imputation: 8


In [ ]:
# Drop the extracted columns
for feature in keywords_dict.keys():
    df.drop(columns=[f'Extracted_{feature}'], inplace=True)

In [ ]:
# Display the first few rows to verify the changes
df[['Public Remarks', 'Building Types', 'Parking Type', 'Ownership Type']].head(10)

,Public Remarks,Building Types,Parking Type,Ownership Type
0,Discover The Perfect Blend Of Artistic Design ...,Apartment,Underground,Condominium
1,"This beautifully maintained 2 bed, 2 bath, cor...",Apartment,Open,Condominium/Strata
2,Presenting a stunning three-story townhouse lo...,Townhouse,Open,Freehold
3,"A 1,240 Sq Ft 3 Bedroom, 2 Bedroom Luxury Cond...",Apartment,Underground,Condominium/Strata
4,Stop searching!! Your lovely home is waiting! ...,Apartment,Street,Condominium/Strata
5,Welcome to the luxurious Tridel Building at Sk...,Apartment,Underground,Condominium/Strata
6,Spectacular Downtown Bloor St. Location! Bloo...,Apartment,Underground,Condominium/Strata
7,Welcome First Time Homebuyers and Investor whe...,Apartment,Underground,Condominium/Strata
8,"Luxury Condo By Daniels 'The Rockefeller ', On...",Apartment,Underground,Condominium/Strata
9,A boutique condo located at the heart of mid-t...,Apartment,Underground,Condominium/Strata


In [ ]:
# Define the extraction function
def extract_amenities(remarks):
    doc = nlp(remarks)
    inhouse_amenities = set()
    nearby_amenities = set()

    # Define some context-based rules
    inhouse_keywords = {'gym', 'pool', 'concierge', 'fireplace', 'parking', 'elevator', 'laundry', 'security', 'garden', 'spa', 'playground', 'bbq', 'lounge', 'party room', 'bike storage', 'storage locker', 'guest suite', 'tennis court', 'basketball court', 'library', 'meeting room', 'theater room', 'golf simulator', 'pet washing station', 'yoga studio'}
    nearby_keywords = {'park', 'subway', 'school', 'market', 'transit', 'shopping', 'mall', 'restaurant', 'cafe', 'hospital', 'bus', 'station', 'store', 'museum'}

    for ent in doc.ents:
        if ent.label_ in ['FAC', 'ORG', 'PRODUCT']:
            inhouse_amenities.add(ent.text)
        elif ent.label_ in ['LOC', 'GPE']:
            nearby_amenities.add(ent.text)

    # Additional keyword-based filtering
    for token in doc:
        token_lower = token.text.lower()
        if token_lower in inhouse_keywords:
            inhouse_amenities.add(token.text.capitalize())
        elif token_lower in nearby_keywords:
            nearby_amenities.add(token.text.capitalize())

    return ', '.join(inhouse_amenities), ', '.join(nearby_amenities)

In [ ]:
# Apply the function to extract amenities from the entire Public Remarks column
df['Extracted Inhouse Amenities'], df['Extracted Nearby Amenities'] = zip(*df['Public Remarks'].apply(extract_amenities))


In [ ]:
# Display the first few rows to show the extracted amenities
df[['Public Remarks', 'Extracted Inhouse Amenities', 'Extracted Nearby Amenities']].head(10)


,Public Remarks,Extracted Inhouse Amenities,Extracted Nearby Amenities
0,Discover The Perfect Blend Of Artistic Design ...,"Parking, Whole Foods, Fireplace, Discover The ...","Store, Subway"
1,"This beautifully maintained 2 bed, 2 bath, cor...","Garden, West & North Exposures for Spectacular...","Balcony, Station, Centre Island/Breakfast Bar,..."
2,Presenting a stunning three-story townhouse lo...,"TTC, Sqft W/, Dishwasher,, Spacious Rooftop De...","Laminate, Stove"
3,"A 1,240 Sq Ft 3 Bedroom, 2 Bedroom Luxury Cond...","Ceilings & Floor/Ceiling Windows, Lounge, Pool...",
4,Stop searching!! Your lovely home is waiting! ...,"TTC, Washer & Dry, Outdoor Terrace, Waterfront...","Distillery District, West Donlands, Market, Sh..."
5,Welcome to the luxurious Tridel Building at Sk...,"Parking, Washer/Dryer, Fireplace, Tridel Build...",Stove
6,Spectacular Downtown Bloor St. Location! Bloo...,"Parking, Washer/Dryer, Lounge, Bbq, Pool, Unde...","Shopping, Subway, Stove"
7,Welcome First Time Homebuyers and Investor whe...,"Minutes Walk, Subway, Scarborough Town Center,...","TTC, Plaza, Schools, Subway, Park, Bus"
8,"Luxury Condo By Daniels 'The Rockefeller ', On...","Parking, Security, Pool, Bayview Subway Statio...","the Bayview Village, Station, Subway, Shopping"
9,A boutique condo located at the heart of mid-t...,"Parking, Concierge, Gym","Station, Subway, Toronto"


In [ ]:
# Function to update existing amenities columns if missing values
def update_amenities(existing_amenities, extracted_amenities):
    if pd.isnull(existing_amenities) or existing_amenities == '':
        return extracted_amenities
    return existing_amenities

In [ ]:
# Update the 'Inhouse Ammenities' column
df['Inhouse Ammenities'] = df.apply(
    lambda row: update_amenities(row['Inhouse Ammenities'], row['Extracted Inhouse Amenities']),
    axis=1
)

# Update the 'Nearby Amenities' column
df['Nearby Amenities'] = df.apply(
    lambda row: update_amenities(row['Nearby Amenities'], row['Extracted Nearby Amenities']),
        axis=1
)

In [ ]:
# Drop the extracted columns
df.drop(columns=['Extracted Inhouse Amenities', 'Extracted Nearby Amenities'], inplace=True)

In [ ]:
# Check for null values in 'Inhouse Ammenities' and 'Nearby Amenities' columns after updating
null_inhouse_amenities_after = df['Inhouse Ammenities'].isnull().sum()
null_nearby_amenities_after = df['Nearby Amenities'].isnull().sum()

In [ ]:
# Print the null values count after updating
print(f'Null values in Inhouse Ammenities after updating: {null_inhouse_amenities_after}')
print(f'Null values in Nearby Amenities after updating: {null_nearby_amenities_after}')

Null values in Inhouse Ammenities after updating: 0
Null values in Nearby Amenities after updating: 0


In [ ]:
# Display the first few rows to verify the changes
print(df[['Public Remarks', 'Inhouse Ammenities', 'Nearby Amenities']].head(10))

                                      Public Remarks  \
0  Discover The Perfect Blend Of Artistic Design ...   
1  This beautifully maintained 2 bed, 2 bath, cor...   
2  Presenting a stunning three-story townhouse lo...   
3  A 1,240 Sq Ft 3 Bedroom, 2 Bedroom Luxury Cond...   
4  Stop searching!! Your lovely home is waiting! ...   
5  Welcome to the luxurious Tridel Building at Sk...   
6  Spectacular Downtown Bloor St.  Location! Bloo...   
7  Welcome First Time Homebuyers and Investor whe...   
8  Luxury Condo By Daniels 'The Rockefeller ', On...   
9  A boutique condo located at the heart of mid-t...   

                                  Inhouse Ammenities  \
0                            Guest Suite, Party Room   
1  Security/Concierge, Exercise Centre, Sauna, St...   
2  TTC, Sqft W/, Dishwasher,, Spacious Rooftop De...   
3  Exercise Centre, Party Room, Security/Concierg...   
4  Security/Concierge, Exercise Centre, Party Roo...   
5                        Exercise Centre, Party

In [ ]:
# Check for missing values in all columns and their data types
missing_values = df.isnull().sum()
dtypes = df.dtypes

In [ ]:
# Create a DataFrame to display missing values and data types together
missing_values_df = pd.DataFrame({
    'Missing Values': missing_values,
    'Data Type': dtypes
})

In [ ]:
# Display the DataFrame
missing_values_df = missing_values_df[missing_values_df['Missing Values'] > 0]
print(missing_values_df)

                   Missing Values Data Type
Bathrooms                       5   float64
Bedrooms                        3   float64
Storey                        147   float64
Building Types                  7    object
Floor Area                    181   float64
Secondary Contact             168    object
Price                           6   float64
Parking Type                   50    object
Parking Space                 167   float64
Ownership Type                  8    object


In [ ]:
# Impute missing values for object type columns with 'Data Not Available'
df['Building Types'].fillna('Data Not Available', inplace=True)
df['Secondary Contact'].fillna('Data Not Available', inplace=True)
df['Parking Type'].fillna('Data Not Available', inplace=True)
df['Ownership Type'].fillna('Data Not Available', inplace=True)

In [ ]:
# For numerical columns, impute missing values with a specific constant value (e.g., -1)
numerical_columns = ['Bathrooms', 'Bedrooms', 'Storey', 'Floor Area', 'Price', 'Parking Space']

In [ ]:
for column in numerical_columns:
    df[column].fillna(-1, inplace=True)

# Check for remaining missing values
missing_values_after_imputation = df.isnull().sum()

In [ ]:
# Print the remaining missing values count for each column
print('Remaining missing values after imputation:')
print(missing_values_after_imputation)

Remaining missing values after imputation:
MSL Numbers           0
Bathrooms             0
Public Remarks        0
Bedrooms              0
Storey                0
Building Types        0
Floor Area            0
Inhouse Ammenities    0
Primary Contact       0
Secondary Contact     0
Price                 0
Property Type         0
Address               0
Longitude             0
Latitude              0
Parking Type          0
Parking Space         0
Ownership Type        0
Nearby Amenities      0
dtype: int64


In [ ]:

# Save the cleaned dataset to a new CSV file
final_cleaned_file_path = '/content/final_cleaned_dataset_realtor.csv'
df.to_csv(final_cleaned_file_path, index=False)
print(f'Final cleaned dataset saved to {final_cleaned_file_path}')



Final cleaned dataset saved to /content/final_cleaned_dataset_realtor.csv
